In [587]:
from pulp import *
import pandas as pd
import networkx as nx
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
#Heavily influenced by, need to make sure we understand the code so we can use it and not plagerize as a final product
#https://www.kaggle.com/code/itoeiji/solving-tsp-and-vrp-by-mip-using-pulp

In [ ]:
#Test Data
distEnum = {
    "SFO": { "SFO": 0, "LAX": 500, "SJC": 75, "OAK": 50},
    "LAX": { "SFO": 500, "LAX": 0, "SJC": 280, "OAK": 800},
    "SJC": { "SFO": 75, "LAX": 280, "SJC": 0, "OAK": 30},
    "OAK": { "SFO": 50, "LAX": 800, "SJC": 30, "OAK": 0}
}

distDict = pd.DataFrame( 
    [
    [0, 500, 75, 50],
    [500, 0, 280, 800],
    [75, 280, 0, 30],
    [50, 800, 30, 0]
    ])

distDict2 = pd.DataFrame( 
    [
    [0, 9999, 469, 857, 375],  #MSY
    [9999, 0, 549, 1188, 464], #IND
    [469, 549, 0, 639, 380],   #ATL
    [857, 1188, 639, 0, 9999], #FLL
    [375, 464, 380, 9999, 0]   #MEM
    ])

In [588]:
usRoutes = pd.read_csv('../data/usRoutesClean.csv')

In [589]:
def findRoutes (dataSet, startAirport, endAirport, cutoff):
    # Create the network so that we can get our additional routes that could be used to get from our start to finish
    G = nx.from_pandas_edgelist(dataSet, 'SRCIATA', 'DESTINIATA', create_using=nx.DiGraph())

    cutoffValue = 0     # Used to track what depth the while loop is at
    noRoute = True      # Used to track if a route was found and the loop can be stopped early

    while (cutoffValue <= cutoff) & (noRoute == True):  # Loop goes through and finds the shallowest depth that we can work with
        cutoffValue = cutoffValue + 1
        paths = nx.all_simple_paths(G, startAirport, endAirport, cutoff=cutoffValue)

        for path in paths:
            noRoute = False

    if cutoffValue > cutoff:
        return(pd.DataFrame())

    paths = nx.all_simple_paths(G, startAirport, endAirport, cutoff=cutoffValue)

    shortestTotalDistance = 0   # Used to track what the shortest total distance is for later comparison of ranking 

    for path in paths: 
        nodeLocation = 0
        totalDistance = 0
        potentialRoute = pd.DataFrame({"SRC": [], "DESTIN": [], "METERS": []})

        for index in path:
            # Calculate total distance of the route
            currentIATA = index
            if nodeLocation != 0:
                distance = (dataSet.loc[(dataSet['SRCIATA'] == pastIATA) & (dataSet['DESTINIATA'] == currentIATA), 'meters'].values)[0]
                totalDistance = totalDistance + distance
                indexEntry = pd.DataFrame({"SRC": [pastIATA], "DESTIN": [currentIATA], "METERS": [distance]})
                potentialRoute = pd.concat([potentialRoute, indexEntry])

            nodeLocation = nodeLocation + 1
            pastIATA = index
        
        if shortestTotalDistance == 0:                  # Takes the first value found a promotes it to the shortest
            shortestTotalDistance = totalDistance
            shortestRoute = potentialRoute.copy()
        else:
            if shortestTotalDistance > totalDistance:   # Compares the shortest with the current to see if it needs to replace
                shortestTotalDistance = totalDistance
                shortestRoute = potentialRoute.copy()

    return(shortestRoute)

In [673]:
def sampleRandomAirports(n):
    sampleAirports = pd.Series(usRoutes['SRCIATA'].sample(n))
    i = 0
    while (i != len(sampleAirports)):
        j = 0
        while (j <= len(sampleAirports) - 1):
            route = findRoutes(usRoutes, sampleAirports.iloc[i], sampleAirports.iloc[j], 50)
            if (route.empty):
                pass
            elif (route['DESTIN'].iloc[0] != sampleAirports.iloc[j]):
                newDestin = pd.Series(route['DESTIN'].iloc[0])
                if(pd.concat([sampleAirports, newDestin]).is_unique):  
                    sampleAirports = pd.concat([sampleAirports, newDestin]) 
            j = j + 1
        i = i + 1
    return sampleAirports

In [645]:
def createDistanceMatrix(sampleAirports):
    DistanceMatrix = []
    i = 0
    while (i != len(sampleAirports)):
        DISTANCEList = []
        j = 0
        while (j <= len(sampleAirports) - 1):
            route = findRoutes(usRoutes, sampleAirports.iloc[i], sampleAirports.iloc[j], 2)

            if (sampleAirports.iloc[i] == sampleAirports.iloc[j]):
                DISTANCEList.append(0)
            elif ((route.empty) or (route['DESTIN'].iloc[0] != sampleAirports.iloc[j])):
                DISTANCEList.append(99999)
            elif (route['DESTIN'].iloc[0] != sampleAirports.iloc[j]):
                DISTANCEList.append(99999)
            else:
                DISTANCEList.append((route['METERS'].iloc[0]/1609.344))
            
            j = j + 1
        DistanceMatrix.append(DISTANCEList)
        i = i + 1

    DistanceMatrix = pd.DataFrame(DistanceMatrix, columns = sampleAirports, index = sampleAirports)
    DistanceMatrix = DistanceMatrix.fillna(0)
    DistanceMatrix = DistanceMatrix.round(0)
    DistanceMatrix = DistanceMatrix.astype(int)
    return DistanceMatrix

In [ ]:
#Change the number in sampleRandomAirports
sampleAirports = sampleRandomAirports(25)
distMatrix = createDistanceMatrix(sampleAirports)
print(distMatrix)

SRCIATA    STL    DFW    BKG    CMH    BOI    MSY    MYR    PDX    BNA    ATL  \
SRCIATA                                                                         
STL          0    634  99999    432  99999    675  99999  99999    325    574   
DFW        635      0  99999   1065  99999    513   1164   2008    689    808   
BKG      99999  99999      0  99999  99999  99999  99999  99999  99999  99999   
CMH        431   1067  99999      0  99999  99999  99999  99999    391    583   
BOI      99999  99999  99999  99999      0  99999  99999    425  99999  99999   
MSY        675    512  99999  99999  99999      0  99999  99999    546    469   
MYR      99999   1164  99999  99999  99999  99999      0  99999  99999    366   
PDX      99999   2008  99999  99999    426  99999  99999      0  99999   2603   
BNA        326    690  99999    389  99999    545  99999  99999      0    255   
ATL        576    807  99999    583  99999    470    366   2604    257      0   
DFW        635      0  99999

In [693]:
distMatrixHeaders = distMatrix
distMatrix = distMatrix.reset_index(drop=True)

columns = []
for i in range(len(distMatrix)):
    columns.append(i)

distMatrix.columns = columns

In [694]:
problem = LpProblem("LP_Opt_AC", LpMinimize)

coordList = []
for i in range(len(distMatrix)):
    for j in range(len(distMatrix)):
        coordList.append((i,j))

x = LpVariable.dicts('x', coordList, lowBound=0, upBound=1, cat='Binary')

problem += pulp.lpSum(distMatrix[i][j] * x[i, j] for i in range(len(distMatrix)) for j in range(len(distMatrix)))

indexes = []
for i in range(len(distMatrix)):
    indexes.append(i)

u = pulp.LpVariable.dicts('u', indexes, lowBound=1, upBound=99999, cat='Integer')

for i in range(len(distMatrix)):
    problem += x[i, i] == 0

for i in range(len(distMatrix)):
    problem += pulp.lpSum(x[i, j] for j in range(len(distMatrix))) == 1
    problem += pulp.lpSum(x[j, i] for j in range(len(distMatrix))) == 1

for i in range(len(distMatrix)):
    for j in range(len(distMatrix)):
        if i != j and (i != 0 and j != 0):
            problem += u[i] - u[j] <= len(distMatrix) * (1 - x[i, j]) - 1

In [695]:
status = problem.solve()
status, LpStatus[status], value(problem.objective)

(1, 'Optimal', 410720.0)

In [696]:
airports_csv = pd.read_csv('../data/openFlightsRaw/airports.csv')

sampleAirportCoords = airports_csv[airports_csv['IATA'].isin(sampleAirports)]

In [697]:
fig = go.Figure(data=go.Scattergeo(
    name = "Has Outgoing Traffic",
    lat = sampleAirportCoords['LAT'].tolist(),
    lon = sampleAirportCoords['LONG'].tolist(), 
    text = sampleAirportCoords['NAME'].tolist(),
    mode = 'markers',
    marker = dict(
        size = 7,
        color = 'black'
    )
))

fig.update_layout(
    title = {
        'text': "Airports from Sample",
        'x': 0.5,
        'xanchor': 'center'
    },
    showlegend=False,
    geo=dict(
        scope = 'usa',
        showland = True,
        landcolor = 'lightgray',
    )
)

fig.show()

In [698]:
valMatrix = []
for i in range(len(distMatrix)):
    valList = []
    for j in range(len(distMatrix)):
        valList.append(int(pulp.value(x[i, j])))
    valMatrix.append(valList)

valMatrix = pd.DataFrame(valMatrix, columns = distMatrixHeaders.columns, index = distMatrixHeaders.index)
    
path = []
for index, row in valMatrix.iterrows():
    matchingCols = row[row == 1].index.tolist()
    for col in matchingCols:
        path.append((index, col))
    
srcLatList = []
srcLongList = []
destinLatList = []
destinLongList = []
for i in range(len(path)):
    src = airports_csv[airports_csv['IATA'] == path[i][0]]
    src = src.reset_index(drop=True)
    destin = airports_csv[airports_csv['IATA'] == path[i][1]]
    destin = destin.reset_index(drop=True)
    srcLatList.append(src.at[0, 'LAT'])
    srcLongList.append(src.at[0, 'LONG'])
    destinLatList.append(destin.at[0, 'LAT'])
    destinLongList.append(destin.at[0, 'LONG'])

pathCoords = pd.DataFrame({
    'SRCLAT': srcLatList,
    'SRCLONG': srcLongList,
    'DESTINLAT': destinLatList,
    'DESTINLONG': destinLongList
})
pathCoords

,SRCLAT,SRCLONG,DESTINLAT,DESTINLONG
0,38.748697,-90.370003,39.998001,-82.891899
1,32.896801,-97.038002,32.896801,-97.038002
2,36.532082,-93.200544,41.504101,-74.104797
3,39.998001,-82.891899,36.124500,-86.678200
4,43.564400,-116.223000,37.618999,-122.375000
5,29.993401,-90.258003,32.896801,-97.038002
6,33.679699,-78.928299,38.852100,-77.037697
7,45.588699,-122.598000,43.564400,-116.223000
8,36.124500,-86.678200,36.124500,-86.678200
9,33.636700,-84.428101,33.679699,-78.928299


In [700]:
fig = go.Figure(data=go.Scattergeo(
    lat = sampleAirportCoords['LAT'].tolist(),
    lon = sampleAirportCoords['LONG'].tolist(), 
    text = sampleAirportCoords['IATA'].tolist(),
    mode = 'markers+text',
    marker = dict(
        size = 7,
        color = 'black'
    ),
    textposition='top center',
    textfont = dict(
            weight = 'bold',
            size = 8
        ),
    )
)

for index, row in pathCoords.iterrows():
    fig.add_trace(go.Scattergeo(
        mode = "lines",
        lat = [row['SRCLAT'], row['DESTINLAT']],
        lon = [row['SRCLONG'], row['DESTINLONG']],
        opacity=.5,
        marker = dict(
            size = 10,
            color = 'red'
        )
    )
)

fig.update_layout(
    title = {
        'text': "Route Created from Sampled Airports",
        'x': 0.5,
        'xanchor': 'center'
    },
    showlegend=False,
    geo=dict(
        scope = 'world',
        showland = True,
        landcolor = 'lightgray',
    )
)

fig.show()